In [1]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("C:\\Users\\Ansh\\Downloads\\tweets.csv")
# here i am printing fisrt five line of dataset
data.head()

,_key,tweet_author,tweet_text
0,1374140386071961602,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...
1,1374032432173842437,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...
2,1373902876553048065,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...
3,1373656782367813635,Toby Eyre,#acalabrutinib is a valuable option in pts int...
4,1372941634334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...


In [3]:
data.shape

(43347, 3)

In [4]:
data.describe()

,_key
count,4.334700e+04
mean,1.003292e+18
std,2.313927e+17
min,5.505794e+17
25%,8.006620e+17
50%,1.011270e+18
75%,1.204047e+18
max,1.374141e+18


In [5]:
data.isna().sum()

_key            0
tweet_author    0
tweet_text      0
dtype: int64

In [6]:
data.head()

,_key,tweet_author,tweet_text
0,1374140386071961602,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...
1,1374032432173842437,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...
2,1373902876553048065,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...
3,1373656782367813635,Toby Eyre,#acalabrutinib is a valuable option in pts int...
4,1372941634334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...


In [7]:
def edits1(word):
    letters='abcdefghijklmnopqrstuvwxyz'
    splits=[(word[:i], word[i:]) for i in range(len(word)+1)]
    deletes=[L+R[1:] for L,R in splits if R]
    transposes=[L+R[1] +R[0] + R[2:] for L,R in splits if len(R)>1]
    replaces = [L+c+R[1:] for L,R in splits if R for c in letters]
    inserts = [L+c+ R for L,R in splits for c in letters]
    return set(deletes+transposes+replaces+inserts)
def edits2(word):
    return(e2 for e1 in edits1(word) for e2 in edits1(e1))
            

In [8]:
def remove_spaces(text):
    text=text.strip()
    text=text.split()
    return ' '.join(text)

In [9]:
contraction = {'cause':'because',
              'aint': 'am not',
              'aren\'t': 'are not'}

def mapping_replacer(x,dic):
    for words in dic.keys():
        if ' ' + words + ' ' in x:
            x=x.replace(' '+ words +' ' ,' '+dic[words]+' ' )
    return x

In [10]:
import nltk
nltk.download('punkit')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

nltk.LancasterStemmer
ls = LancasterStemmer()
lem = WordNetLemmatizer()
def lexicon_normalization(text):
    words = word_tokenize(text) 
    
    
    # 1- Stemming
    words_stem = [ls.stem(w) for w in words]
    
    # 2- Lemmatization
    words_lem = [lem.lemmatize(w) for w in words_stem]
    return words_lem


[nltk_data] Error loading punkit: Package 'punkit' not found in index


In [11]:
import emoji
import re 
#from emot.emo_unicode import UNICODE_EMO
def convert_emojis(text):
    for emot in emoji.UNICODE_EMOJI:
        text = re.sub(r'('+emot+')', "_".join(emoji.UNICODE_EMOJI[emot].replace(",","").replace(":","").split()), text)
    return text

In [12]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\'','', text)
    
    return text

In [13]:
from collections import Counter
def remove_stopword(text):
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])
    return text

In [14]:
def tokenise(text):
    words = word_tokenize(text) 
    return words

In [25]:
import re
data['tweet_text'] = data['tweet_text'].map(lambda x: re.sub(r'\W+', ' ', x))
data['tweet_text'] = data['tweet_text'].replace(r'\W+', ' ', regex=True)

In [26]:
data.tweet_text

0         Scientists conducted a Phase II study of acal...
1        This phase 2 Acalabrutinib Venetoclax AV trial...
2         NICE backs AstraZenecas Calquence for CLL htt...
3         acalabrutinib is a valuable option in pts int...
4        NICE has recommended the use of acalabrutinib ...
                               ...                        
43342    Hanging out with Friends FF CLL Happiness http...
43343    Hanging out with Friends FF CLL Happiness http...
43344    Zusatznutzen von Idelalisib ist weder für CLL ...
43345     Hematología PTK2 EXPRESSION AND IMMUNOCHEMOTH...
43346     Hematología MUTATIONS IN TLR MYD88 PATHWAY ID...
Name: tweet_text, Length: 43347, dtype: object

In [27]:
data.head()

,_key,tweet_author,tweet_text
0,1374140386071961602,Hematopoiesis News,Scientists conducted a Phase II study of acal...
1,1374032432173842437,Michael Wang MD,This phase 2 Acalabrutinib Venetoclax AV trial...
2,1373902876553048065,1stOncology,NICE backs AstraZenecas Calquence for CLL htt...
3,1373656782367813635,Toby Eyre,acalabrutinib is a valuable option in pts int...
4,1372941634334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...


In [58]:
df = data.groupby(by=['tweet_author']).count()


In [59]:
df.rename(columns = {'tweet_author':'Author', 'tweet_text':''}, inplace = True)

In [60]:
df

,_key,
tweet_author,,
,41,41
Adam Grant,1,1
AiMS,3,3
Anyii Vega,1,1
AplusA_France,10,10
...,...,...
𝗡𝗶𝗵𝗮𝗿 𝗗𝗲𝘀𝗮𝗶 𝗠𝗗,3,3
𝘮𝘦𝘭ᶜ,1,1
𝙅𝙤𝙝𝙣 𝙒𝙖𝙡𝙡,1,1


In [61]:
data['tweet_text']=data['tweet_text'].apply(lambda x: mapping_replacer(x, contraction))

In [62]:
data['tweet_text'] = data['tweet_text'].apply(lambda x:clean_text(x))

In [63]:
data['tweet_text']=data['tweet_text'].apply(lambda x: remove_stopword(x))

In [64]:
data['tweet_text']=data['tweet_text'].apply(lambda x: lexicon_normalization(x))

In [65]:
data.head()

,_key,tweet_author,tweet_text
0,1374140386071961602,Hematopoiesis News,"[sci, conduc, phas, ii, study, acalabrutinib, ..."
1,1374032432173842437,Michael Wang MD,"[phas, acalabrutinib, venetoclax, av, tri, sti..."
2,1373902876553048065,1stOncology,"[nic, back, astrazeneca, calqu, cll, http, co]"
3,1373656782367813635,Toby Eyre,"[acalabrutinib, valu, opt, pt, intol, ibrutini..."
4,1372941634334232586,Lymphoma Hub,"[nic, recommend, u, acalabrutinib, paty, tre, ..."


## Polarity Of Sentiment


In [66]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator

from textblob import TextBlob

def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(tweet) 
    
    # set sentiment 
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [67]:
data['sentiment']=data['tweet_text'].apply(lambda x: get_tweet_sentiment(' '.join(x)))

In [74]:
data.to_csv(path_or_buf='C:\\Users\\Ansh\\Desktop\\data.csv')

In [68]:
Positive_sent = data[data['sentiment']=='positive']
Negative_sent = data[data['sentiment']=='negative']
Neutral_sent = data[data['sentiment']=='neutral']

In [69]:
print('Number of tweets with positive sentiment', Positive_sent['sentiment'].count())
print('Number of tweets with negative sentiment', Negative_sent['sentiment'].count())
print('Number of tweets with neutral sentiment', Neutral_sent['sentiment'].count())

Number of tweets with positive sentiment 12832
Number of tweets with negative sentiment 3207
Number of tweets with neutral sentiment 27308


#### Here we got to know that for finding out the polarity we need to clean the data, remove stopwords and tokenise the tweets

In [ ]:
top = Counter([item for sublist in data['tweet_text'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')


In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
top = Counter([item for sublist in data['tweet_text'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

In [ ]:
fig = px.treemap(temp, path=['Common_words'], values='count',title='Tree of Most Common Words')
fig.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator

from textblob import TextBlob

def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(tweet) 
    
    # set sentiment 
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [ ]:
data['sentiment']=data['tweet_text'].apply(lambda x: get_tweet_sentiment(' '.join(x)))

In [ ]:
data

In [ ]:
Positive_sent = data[data['sentiment']=='positive']
Negative_sent = data[data['sentiment']=='negative']
Neutral_sent = data[data['sentiment']=='neutral']

In [ ]:
print('Number of tweets with positive sentiment', Positive_sent['sentiment'].count())
print('Number of tweets with negative sentiment', Negative_sent['sentiment'].count())
print('Number of tweets with neutral sentiment', Neutral_sent['sentiment'].count())

In [ ]:
#MosT common positive words
top = Counter([item for sublist in Positive_sent['tweet_text'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Common_words','count']
temp_positive.style.background_gradient(cmap='Greens')

In [ ]:

import numpy as np
top = Counter([item for sublist in Positive_sent['tweet_text'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(23))
temp_positive.columns = ['Common_words','count']
temp_positive['Common_words'] = temp_positive['Common_words'].map(lambda x: re.sub(r'\W+', '', x))
temp_positive['Common_words'] = temp_positive['Common_words'].replace(r'\W+', '', regex=True)
temp_positive['Common_words'] = temp_positive['Common_words'].apply(lambda x:remove_spaces(x))
temp_positive=temp_positive[~temp_positive['Common_words'].isin(['s','gre','“',' * '])] #new line removing meaningless words
mask1 = temp_positive.Common_words.str.contains('[a-zA-Z]')
mask2 = temp_positive.Common_words.notna()
temp_positive = temp_positive[mask1 | mask2]
temp_positive.Common_words =  temp_positive.Common_words.str.replace(r"\s+", "").replace("", np.NaN)
temp_positive=temp_positive.dropna()


temp_positive.style.background_gradient(cmap='Greens')

In [ ]:
fig = px.bar(temp_positive, x="count", y="Common_words", title='Most Commmon Words in Positive Sentiment tweets', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
#MosT common negative words
top = Counter([item for sublist in Negative_sent['tweet_text'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']
temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
#MosT common negative words
top = Counter([item for sublist in Negative_sent['tweet_text'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(22))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']

#Data cleaning
temp_negative['Common_words'] = temp_negative['Common_words'].map(lambda x: re.sub(r'\W+', '', x))
temp_negative['Common_words'] = temp_negative['Common_words'].replace(r'\W+', '', regex=True)
temp_negative=temp_negative[~temp_negative['Common_words'].isin(['s','t'])] #new line removing meaningless words from above
#mask1 = temp_negative.Common_words.str.contains('[a-zA-Z]')
#mask2 = temp_negative.Common_words.notna()
#temp_negative = temp_negative[mask1 | mask2]

temp_negative.Common_words =  temp_negative.Common_words.replace("", np.nan)
temp_negative = temp_negative.dropna(subset=['Common_words'])

temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
fig = px.treemap(temp_negative, path=['Common_words'], values='count',title='Tree Of Most Common Words in Negative Tweets')
fig.show()

In [ ]:
#MosT common Neutral words
top = Counter([item for sublist in Neutral_sent['tweet_text'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='YlOrBr')

In [ ]:
top = Counter([item for sublist in Neutral_sent['tweet_text'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']

#Data cleaning
temp_neutral['Common_words'] = temp_neutral['Common_words'].map(lambda x: re.sub(r'\W+', '', x))
temp_neutral['Common_words'] = temp_neutral['Common_words'].replace(r'\W+', '', regex=True)
temp_neutral=temp_neutral[~temp_neutral['Common_words'].isin(['s'])] #new line removing meaningless words from above

temp_neutral.Common_words =  temp_neutral.Common_words.replace("", np.nan)
temp_neutral = temp_neutral.dropna(subset=['Common_words'])

temp_neutral.style.background_gradient(cmap='YlOrBr')

In [ ]:
fig = px.bar(temp_neutral, x="count", y="Common_words", title='Most Commmon Neutral Words', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
fig = px.treemap(temp_neutral, path=['Common_words'], values='count',title='Tree Of Most Common Neutral Words')
fig.show()

In [ ]:
raw_text = [word for word_list in data['tweet_text'] for word in word_list]

In [ ]:
def words_unique(sentiment,numwords,raw_words):
    '''
    Input:
        segment - Segment category (ex. 'Neutral');
        numwords - how many specific words do you want to see in the final result; 
        raw_words - list  for item in train_data[train_data.segments == segments]['temp_list1']:
    Output: 
        dataframe giving information about the name of the specific ingredient and how many times it occurs in the chosen cuisine (in descending order based on their counts)..

    '''
    allother = []
    for item in data[data.sentiment != sentiment]['tweet_text']:
        for word in item:
            allother .append(word)
    allother  = list(set(allother ))
    
    specificnonly = [x for x in raw_text if x not in allother]
    
    mycounter = Counter()
    
    for item in data[data.sentiment == sentiment]['tweet_text']:
        for word in item:
            mycounter[word] += 1
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['words','count'])
    
    return Unique_words

In [ ]:
Unique_Positive= words_unique('positive', 20, raw_text)
print("The top 20 unique words in Positive Tweets are:")
Unique_Positive.style.background_gradient(cmap='Greens')

In [ ]:
fig = px.treemap(Unique_Positive, path=['words'], values='count',title='Tree Of Unique Words in Positive sentiment tweets')
fig.show()

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(Unique_Positive['count'], labels=Unique_Positive.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('Donut Plot Of Unique words in Positive sentiment tweets')
plt.show()

In [ ]:
Unique_Negative= words_unique('negative', 10, raw_text)
print("The top 10 unique words in Negative Tweets are:")
Unique_Negative.style.background_gradient(cmap='Reds')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.rcParams['text.color'] = 'black'
plt.pie(Unique_Negative['count'], labels=Unique_Negative.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique words in Negative sentiment tweets')
plt.show()

In [ ]:
Unique_Neutral= words_unique('neutral', 10, raw_text)
print("The top 10 unique words in Neutral Tweets are:")
Unique_Neutral.style.background_gradient(cmap='YlOrBr')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(Unique_Neutral['count'], labels=Unique_Neutral.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique words in Neutral sentiment tweets')
plt.show()

# Word Cloud 
### For Positive, Neutral, Negative Sentiments

In [ ]:
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), color = 'white',
                   title = None, title_size=40, image_color=False):
    
    wordcloud = WordCloud(background_color=color,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=400, 
                    height=200,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  

In [ ]:
plot_wordcloud(Neutral_sent.tweet_text,color='white',max_font_size=100,title_size=30,title="WordCloud of Neutral Tweets")

In [ ]:
plot_wordcloud(Positive_sent.tweet_text,title="Word Cloud Of Positive tweets",title_size=30)

In [ ]:
plot_wordcloud(Negative_sent.tweet_text,title="Word Cloud Of Negative tweets",title_size=30)